In [ ]:
%matplotlib inline

#Import modules
import datetime
import os
import pathlib
import sqlite3
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#import csv
import glob as gb

In [ ]:
mpl.rcParams['figure.figsize'] = (16, 9)
pd.options.display.max_rows = 200

In [ ]:
gb.glob('**/*.sql', recursive=True)

In [ ]:
class EPLusSQL():

    def __init__(self, sql_path = None):
        abs_sql_path = os.path.abspath(sql_path)
        self.sql_uri = '{}?mode=ro'.format(pathlib.Path(abs_sql_path).as_uri())
        
    def get_annual_energy_by_fuel_and_enduse(self):
        """
        Queries SQL file and returns the ABUPS' End Uses table

        Parameters
        ----------
        None

        Returns
        -------
        df_end_use: pd.DataFrame
            Annual End Use table
            index = 'EndUse'
            columns = ['FuelType','Units']
        """


        # RowName = '#{end_use}'
        # ColumnName='#{fuel_type}'
        annual_end_use_query = """SELECT RowName, ColumnName, Units, Value
            FROM TabularDataWithStrings
            WHERE ReportName='AnnualBuildingUtilityPerformanceSummary'
            AND ReportForString='Entire Facility'
            AND TableName='End Uses'
        """

        with sqlite3.connect(self.sql_uri, uri=True) as con:
            df_end_use = pd.read_sql(annual_end_use_query, con=con)

        # Convert Value to Float
        df_end_use['Value'] = pd.to_numeric(df_end_use['Value'])

        df_end_use = df_end_use.set_index(['RowName',
                                         'ColumnName',
                                         'Units'])['Value'].unstack([1, 2])
        df_end_use.index.name = 'EndUse'
        df_end_use.columns.names = ['FuelType', 'Units']

        end_use_order = ['Heating', 'Cooling',
                         'Interior Lighting', 'Exterior Lighting',
                         'Interior Equipment', 'Exterior Equipment',
                         'Fans', 'Pumps', 'Heat Rejection', 'Humidification',
                         'Heat Recovery', 'Water Systems',
                         'Refrigeration', 'Generators']
        col_order = [
            'Electricity', 'Natural Gas', 'Gasoline', 'Diesel', 'Coal',
            'Fuel Oil No 1', 'Fuel Oil No 2', 'Propane', 'Other Fuel 1',
            'Other Fuel 2', 'District Cooling', 'District Heating',
            'Water']
        df_end_use = df_end_use[col_order].loc[end_use_order]

        # Filter out columns with ALL zeroes
        df_end_use = df_end_use.loc[:,(df_end_use > 0).any(axis=0)]

        return df_end_use


    def get_unmet_hours_table(self):
        """
        Queries 'SystemSummary' and returns all unmet hours

        Parameters
        ----------
        None

        Returns
        -------
        df_unmet: pd.DataFrame
            A DataFrame where


        """

        query = """SELECT RowName, ColumnName, Units, Value FROM TabularDataWithStrings
    WHERE ReportName='SystemSummary'
    AND ReportForString='Entire Facility'
    AND TableName='Time Setpoint Not Met'
    """
        with sqlite3.connect(self.sql_uri, uri=True) as con:
            df_unmet = pd.read_sql(query, con=con)

        # Convert Value to Float
        df_unmet['Value'] = pd.to_numeric(df_unmet['Value'])

        df_unmet = df_unmet.pivot(index='RowName',
                                  columns='ColumnName',
                                  values='Value')

        df_unmet.columns.names = ['Time Setpoint Not Met (hr)']

        # Move 'Facility' as last row (Should always be in the index...)
        if 'Facility' in df_unmet.index:
            df_unmet = df_unmet.loc[[x for x
                                     in df_unmet.index
                                     if x != 'Facility'] + ['Facility']]

        return df_unmet
    
    def get_reporting_vars(self):
        """
        Queries 'ReportingDataDictionary' and returns a DataFrame

        Parameters
        -----------
        None

        Returns
        ---------
        df_vars: pd.DataFrame
            A DataFrame where each row is a reporting variable
        """

        with sqlite3.connect(self.sql_uri, uri=True) as con:
            query = '''
        SELECT KeyValue, Name, TimestepType, ReportingFrequency, Units, Type
            FROM ReportDataDictionary
            '''
            df_vars = pd.read_sql(query, con=con)

        return df_vars
    
    
    def get_hourly_variables(self, variables_list):
        """
        Queries Hourly variables which names are in variables_list
        
        eg: variables_list=['Zone Thermal Comfort CEN 15251 Adaptive Model Temperature']
        """
        
        query = '''
        SELECT EnvironmentPeriodIndex, Month, Day, Hour, Minute,
            ReportingFrequency, KeyValue, Name, Units,
            Value
        FROM ReportVariableWithTime
        '''

        cond = []

        cond.append(
            ("UPPER(Name) IN ({})".format(', '.join(
                              map(repr, [name.upper() for name in variables_list]))))
        )

        cond.append('ReportingFrequency = "Hourly"')

        query += '  WHERE {}'.format('\n    AND '.join(cond))

        with sqlite3.connect(self.sql_uri, uri=True) as con:
            df = pd.read_sql(query, con=con)

        df_pivot = pd.pivot_table(df, values='Value',
                                  columns=['ReportingFrequency', 'KeyValue',
                                           'Name', 'Units'],
                                  index=['EnvironmentPeriodIndex',
                                         'Month', 'Day', 'Hour', 'Minute'])

        df_pivot = df_pivot.loc[3] # Get the annual environment period index

        # We know it's hourly, so recreate a clear index
        (month, day, hour, minute) = df_pivot.index[0]
        start = datetime.datetime(2005, month, day)
        df_pivot.index = pd.date_range(start=start, periods=df_pivot.index.size, freq='H')
        df_pivot = df_pivot['Hourly']

        return df_pivot
    
    def get_timestep_variables(self, variables_list = None):
        """
        Queries 'Zone Timestep' variables which names are in variables_list (if supplied, otherwise all)
        
        eg: variables_list=['Zone Thermal Comfort CEN 15251 Adaptive Model Temperature']
        """
        
        query = '''
        SELECT EnvironmentPeriodIndex, Month, Day, Hour, Minute,
            ReportingFrequency, KeyValue, Name, Units,
            Value
        FROM ReportVariableWithTime
        '''

        cond = []

        if variables_list:
            cond.append(
                ("UPPER(Name) IN ({})".format(', '.join(
                                  map(repr, [name.upper() for name in variables_list]))))
            )

        cond.append('ReportingFrequency = "Zone Timestep"')

        query += '  WHERE {}'.format('\n    AND '.join(cond))

        with sqlite3.connect(self.sql_uri, uri=True) as con:
            df = pd.read_sql(query, con=con)

        df_pivot = pd.pivot_table(df, values='Value',
                                  columns=['ReportingFrequency', 'KeyValue',
                                           'Name', 'Units'],
                                  index=['EnvironmentPeriodIndex',
                                         'Month', 'Day', 'Hour', 'Minute'])

        df_pivot = df_pivot.loc[3] # Get the annual environment period index

        # We know it's Zone Timestep, with 15min timestep, so recreate a clear index
        (month, day, hour, minute) = df_pivot.index[0]
        start = datetime.datetime(2005, month, day)
        
        df_pivot.index = pd.date_range(start=start, periods=df_pivot.index.size, freq='15Min')
        df_pivot = df_pivot['Zone Timestep']

        return df_pivot

## Energy Use by Fuel and End Use

In [ ]:
eplus_sql = EPLusSQL(sql_path='OS/Office_BXL/run/eplusout.sql')

In [ ]:
df_end_use = eplus_sql.get_annual_energy_by_fuel_and_enduse()
df_end_use.applymap(lambda x: x if x > 0 else '')

## Unmet Hours

In [ ]:
df_unmet = eplus_sql.get_unmet_hours_table()
df_unmet

## Look at available reporting variables

In [ ]:
df_vars = eplus_sql.get_reporting_vars()
df_vars

In [ ]:
df_vars['Name'].unique()

## Load comfort variables

In [ ]:
comfort_vars = [
#'Zone Air Temperature',
#'Zone Mean Air Temperature',
#'Zone Mean Radiant Temperature',
'Zone Operative Temperature',
'Zone Thermal Comfort CEN 15251 Adaptive Model Temperature'
]

In [ ]:
df_vars[df_vars['Name'].isin(comfort_vars)]

In [ ]:
df = eplus_sql.get_hourly_variables(variables_list=comfort_vars)

In [ ]:
df.groupby(level=1, axis=1).plot(figsize=(16, 9))

In [ ]:
df.loc[:, pd.IndexSlice[:, 'Zone Thermal Comfort CEN 15251 Adaptive Model Temperature']]

In [ ]:
df.swaplevel(0, 1, axis=1)['Zone Thermal Comfort CEN 15251 Adaptive Model Temperature'].plot()

In [ ]:
df.swaplevel(0, 1, axis=1)['Zone Thermal Comfort CEN 15251 Adaptive Model Temperature'].plot(subplots=True, figsize=(16, 16))

## Look at heat balance

In [ ]:
df = eplus_sql.get_timestep_variables(variables_list=None)

In [ ]:
df.columns.tolist()

In [ ]:
cols = [
('Environment', 'Site Diffuse Solar Radiation Rate per Area', 'W/m2'),
 ('Environment', 'Site Direct Solar Radiation Rate per Area', 'W/m2'),
 ('Environment', 'Site Outdoor Air Drybulb Temperature', 'C'),
 ('MID_OFFICES_S_TZ', 'Zone Mean Air Temperature', 'C'),
 ('MID_S_WINDOW',
  'Surface Outside Face Incident Beam Solar Radiation Rate per Area',
  'W/m2'),
 ('MID_S_WINDOW',
  'Surface Outside Face Incident Sky Diffuse Solar Radiation Rate per Area',
  'W/m2'),
 ('MID_S_WINDOW',
  'Surface Outside Face Incident Solar Radiation Rate per Area',
  'W/m2'),
 ('MID_S_WINDOW', 'Surface Shading Device Is On Time Fraction', ''),
# ('MID_S_WINDOW', 'Surface Storm Window On Off Status', ''),
# ('MID_S_WINDOW', 'Surface Window Blind Slat Angle', 'deg'),
 ('MID_S_WINDOW', 'Surface Window Heat Gain Rate', 'W'),
 ('MID_S_WINDOW', 'Surface Window Heat Loss Rate', 'W'),
 ('MID_S_WINDOW',
  'Surface Window Shading Device Absorbed Solar Radiation Rate',
  'W'),
 ('MID_S_WINDOW', 'Surface Window Transmitted Solar Radiation Rate', 'W'),
]

df = df[cols]

In [ ]:
day = 0 # Which day to plot, zero-indexed
df_day = df.iloc[(4*24)*day + 8*4:(4*24)*day+20*4]

In [ ]:
# Only one image, for sharing...

grouped = df_day.groupby(level='Units', axis=1)

ncols = 1
nrows = int(np.ceil(grouped.ngroups/ncols))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16,16), sharey=False)

for (key, ax) in zip(grouped.groups.keys(), axes.flatten()):
    grouped.get_group(key).plot(ax=ax)

ax.legend()
plt.show()